In [1]:
import glob
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
%matplotlib inline
plt.style.use('ggplot')

/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/nvidia/github/python_environments/soundflux/lib/python3.5/

/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/nvidia/github/python_environments/soundflux/lib/python3.5/

In [3]:
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support
import soundfile

replicating https://github.com/satyen95/Urban-Sound-Classification/blob/master/NN_HL2.ipynb

# Load Sounds

In [4]:
def load_sound_files(file_paths):
    raw_sounds = []
    for fp in file_paths:
        X,sr = soundfile.read(fp,channels=1)
        raw_sounds.append(X)
    return raw_sounds

In [107]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    #tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast#,tonnetz

def parse_audio_files(parent_dir,sub_dirs,file_ext='*.wav'):
    features, labels = np.empty((0,187)), np.empty(0)
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            try:
                #mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
                mfccs, chroma, mel, contrast = extract_feature(fn)
            except Exception as e:
                print(e)
                continue
            #print(fn.split('/'),len(fn.split('/')))
            #ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
            ext_features = np.hstack([mfccs,chroma,mel,contrast])
            features = np.vstack([features,ext_features])
            labels = np.append(labels, fn.split('/')[8].split('-')[1])
    return np.array(features), np.array(labels, dtype = np.int)

def test_parse(file_name):
    features = np.empty((0,187))
    try:
        #mfccs, chroma, mel, contrast,tonnetz = extract_feature(file_name)
        mfccs, chroma, mel, contrast= extract_feature(file_name)
    except Exception as e:
        print(e)
    ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
    features = np.vstack([features,ext_features])
    return np.array(features)


In [44]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

# TEST Features!

In [71]:
X, sample_rate = soundfile.read('/home/nvidia/piano2.wav')
print(X.shape,sample_rate)

(302712, 2) 48000


In [91]:
X[:,:1]

array([[-3.35693359e-04],
       [ 9.15527344e-05],
       [-6.71386719e-04],
       ...,
       [ 3.35693359e-04],
       [ 2.13623047e-04],
       [ 4.57763672e-04]])

In [70]:
X, sample_rate = librosa.load('/home/nvidia/piano2.wav')
print(X.shape,sample_rate)

(139059,) 22050


In [68]:
print(X.shape,sample_rate)

(139059,) 22050


In [98]:
mfccs, chroma, mel, contrasttt= extract_feature('/home/nvidia/piano2.wav')
np.hstack([mfccs,chroma,mel,contrast]).shape

/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)


(187,)

In [35]:
contrast.shape

(7,)

# Actual ones

In [ ]:
parent_dir = '/media/nvidia/d171b53a-0021-4d77-b891-8d723bedc16d1/soundflux/urbansounds/audio'
tr_sub_dirs = ["fold1","fold2"]
ts_sub_dirs = ["fold3"]
tr_features, tr_labels = parse_audio_files(parent_dir,tr_sub_dirs)
ts_features, ts_labels = parse_audio_files(parent_dir,ts_sub_dirs)

tr_labels = one_hot_encode(tr_labels)
ts_labels = one_hot_encode(ts_labels)

/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/util/utils.py:828: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)


/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/util/utils.py:828: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)


/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/util/utils.py:828: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)


/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/util/utils.py:828: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/nvidia/

/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/util/utils.py:828: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)


...

/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/util/utils.py:828: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/nvidia/

/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/util/utils.py:828: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/nvidia/

/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/util/utils.py:828: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)


/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/util/utils.py:828: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/nvidia/

/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/util/utils.py:828: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])
/home/nvidia/github/python_environments/soundflux/lib/python3.5/site-packages/librosa/core/pitch.py:159: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)


In [37]:
training_epochs = 50
n_dim = tr_features.shape[1]
n_classes = 10
n_hidden_units_one = 280 
n_hidden_units_two = 300
sd = 1 / np.sqrt(n_dim)
learning_rate = 0.01

In [38]:
X = tf.placeholder(tf.float32,[None,n_dim])
Y = tf.placeholder(tf.float32,[None,n_classes])

W_1 = tf.Variable(tf.random_normal([n_dim,n_hidden_units_one], mean = 0, stddev=sd))
b_1 = tf.Variable(tf.random_normal([n_hidden_units_one], mean = 0, stddev=sd))
h_1 = tf.nn.tanh(tf.matmul(X,W_1) + b_1)

W_2 = tf.Variable(tf.random_normal([n_hidden_units_one,n_hidden_units_two], 
mean = 0, stddev=sd))
b_2 = tf.Variable(tf.random_normal([n_hidden_units_two], mean = 0, stddev=sd))
h_2 = tf.nn.sigmoid(tf.matmul(h_1,W_2) + b_2)

W = tf.Variable(tf.random_normal([n_hidden_units_two,n_classes], mean = 0, stddev=sd))
b = tf.Variable(tf.random_normal([n_classes], mean = 0, stddev=sd))
y_ = tf.nn.softmax(tf.matmul(h_2,W) + b)

init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [39]:
cost_function = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [41]:
cost_history = np.empty(shape=[1],dtype=float)
y_true, y_pred = None, None
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):            
        _,cost = sess.run([optimizer,cost_function],feed_dict={X:tr_features,Y:tr_labels})
        cost_history = np.append(cost_history,cost)
    
    y_pred = sess.run(tf.argmax(y_,1),feed_dict={X: ts_features})
    y_true = sess.run(tf.argmax(ts_labels,1))
    print("Test accuracy: ",round(session.run(accuracy, 
    	feed_dict={X: ts_features,Y: ts_labels}),3))

fig = plt.figure(figsize=(10,8))
plt.plot(cost_history)
plt.axis([0,training_epochs,0,np.max(cost_history)])
plt.show()

p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average="micro")
print("F-Score:", round(f,3))

ValueError: Cannot feed value of shape (0, 0) for Tensor 'Placeholder_1:0', which has shape '(?, 10)'